<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/VAA_aggresive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# 첫 설치 이후 설치 안함
# pip install yfinance

In [55]:
# 첫 설치 이후 설치 안함
# pip install pandas_market_calendars

In [56]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import pandas as pd
import exchange_calendars as ecals
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)

# ETF 가격 지정된 기간에서 가져와서 DataFrame에 담기
def get_price_data(x, start_date, end_date):
  df_price = pd.DataFrame(columns=x)
  for ticker in x:
    df_price[ticker] = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)['Adj Close']
  return df_price.dropna(axis=0)

# 1년 전 개장일 구하기
us_market_code = "XNYS"
us_calendar = ecals.get_calendar(us_market_code)

# 각 날짜 변수에 저장
today = datetime.today().date()
today_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=0)).date())
month_01_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=30)).date())
month_03_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=91)).date())
month_06_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=182)).date())
month_12_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=365)).date())

# 전략에 사용할 ETF 명 리스트에 담기
VAA_agg = ['SPY', 'VEA', 'VWO', 'AGG', 'SHY', 'IEF', 'LQD']

# ETF 가격 추출
df_VAA_agg = get_price_data(VAA_agg, month_12_ago_workday, today).sort_index(ascending=False)
display(df_VAA_agg)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,SPY,VEA,VWO,AGG,SHY,IEF,LQD
Date,,,,,,,
2023-08-14,448.1100,45.9300,40.6400,96.0000,80.9000,93.9800,105.0100
2023-08-11,445.6500,46.1200,40.8300,96.0700,80.9700,94.1300,105.0500
2023-08-10,445.9100,46.3800,41.4300,96.3700,81.0500,94.6300,105.4100
2023-08-09,445.7500,46.2400,41.3500,97.0000,81.0900,95.3600,106.1600
2023-08-08,448.7500,46.2000,41.2800,96.9300,81.1400,95.3000,106.0600
...,...,...,...,...,...,...,...
2022-08-18,421.0149,41.7882,40.5529,99.7845,80.5913,100.7491,108.6893
2022-08-17,419.7949,41.9244,40.8134,99.5708,80.4742,100.5541,108.4102
2022-08-16,422.7958,42.2843,40.9099,100.1634,80.5425,101.2364,109.4399


In [57]:
# 모멘텀스코어 계산할 오늘-1-3-6-12개월 날짜 저장
ago_name = ['today', 'month_01_ago', 'month_03_ago', 'month_06_ago', 'month_12_ago']
ago_date = [today_workday.date(), \
            month_01_ago_workday.date(), \
            month_03_ago_workday.date(), \
            month_06_ago_workday.date(), \
            month_12_ago_workday.date()]

# '개월 전/날짜' 형식으로 텍스트 저장
ago = [f'{a}/{d}' for a, d in zip(ago_name, ago_date)]

# ETF의 날짜별 데이터 담기
etf_data = {}

for i in range(7):
  etf_data[VAA_agg[i]] = [df_VAA_agg.loc[today_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_01_ago_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_03_ago_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_06_ago_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_12_ago_workday, VAA_agg[i]]]

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
df_etf_price = pd.DataFrame(etf_data, columns=VAA_agg, index=ago)
str_etf_price = str_today + "### ETF 가격 현재-1-3-6-12 개월"

display(Markdown(str_etf_price))
display(df_etf_price)

# csv 경로 설정
output_path =  '/content/drive/MyDrive/colab/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} VAA_agg.xlsx'

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_etf_price.to_csv(f, index=True)

### 2023-08-15 ### ETF 가격 현재-1-3-6-12 개월

,SPY,VEA,VWO,AGG,SHY,IEF,LQD
today/2023-08-14,448.1100,45.9300,40.6400,96.0000,80.9000,93.9800,105.0100
month_01_ago/2023-07-14,449.2800,47.2000,41.9300,97.6743,80.9579,96.2860,107.0824
month_03_ago/2023-05-15,411.4815,46.0248,40.0819,98.3555,81.5598,98.5277,106.6788
month_06_ago/2023-02-13,409.7384,45.0133,41.1859,97.2135,80.1224,95.8645,106.0275
month_12_ago/2022-08-12,420.2376,42.5469,41.0449,100.1829,80.5425,101.0317,109.9403


In [58]:
etf_momentum_score = []
etf_ror_data = []

## DataFrame 만들기: 수익률
for etf_name in VAA_agg:
  etf_today_price = df_etf_price.loc[ago[0], etf_name]
  etf_momentum_score.append((12 * (etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100) + \
                            (4 * (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100) + \
                            (2 * (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100) + \
                            (1 * (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100))

  etf_ror_data.append([(etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100])

df_etf_ror = pd.DataFrame(etf_ror_data, columns=ago[1:], index=VAA_agg).transpose()

str_etf_ror = str_today + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)
  # df_etf_ror.to_csv(filename, mode='a', header=True, index=True)

### 2023-08-15 ### ETF 1-3-6-12 개월 수익률

,SPY,VEA,VWO,AGG,SHY,IEF,LQD
month_01_ago/2023-07-14,-0.2604,-2.6907,-3.0766,-1.7142,-0.0715,-2.3949,-1.9353
month_03_ago/2023-05-15,8.9016,-0.2059,1.3925,-2.3949,-0.8090,-4.6157,-1.5643
month_06_ago/2023-02-13,9.3649,2.0365,-1.3255,-1.2483,0.9705,-1.9658,-0.9596
month_12_ago/2022-08-12,6.6325,7.9515,-0.9866,-4.1752,0.4439,-6.9797,-4.4846


In [59]:
## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 12 if row.name == ago[1] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 4 if row.name == ago[2] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 2 if row.name == ago[3] else row, axis=1)

str_etf_ror_weight = str_today + "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

### 2023-08-15 ### ETF 1-3-6-12 개월 가중치 적용

,SPY,VEA,VWO,AGG,SHY,IEF,LQD
month_01_ago/2023-07-14,-3.1250,-32.2881,-36.9187,-20.5699,-0.8577,-28.7394,-23.2236
month_03_ago/2023-05-15,35.6064,-0.8237,5.5698,-9.5796,-3.2359,-18.4627,-6.2572
month_06_ago/2023-02-13,18.7298,4.0731,-2.6510,-2.4965,1.9409,-3.9317,-1.9192
month_12_ago/2022-08-12,6.6325,7.9515,-0.9866,-4.1752,0.4439,-6.9797,-4.4846


In [60]:
## DataFrame 만들기: 모멘텀
df_etf_momentum_score = pd.DataFrame(etf_momentum_score, columns=['momentum_score'], index=VAA_agg).transpose()

str_etf_momentum_score = str_today + "### ETF 별 12개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### 2023-08-15 ### ETF 별 12개월 Momentum Score

,SPY,VEA,VWO,AGG,SHY,IEF,LQD
momentum_score,57.8437,-21.0873,-34.9864,-36.8213,-1.7087,-58.1134,-35.8846


In [61]:
# 0 이하인 값이 있는지 확인 후 어떤 자산 매수할 지 산출
has_negative_values = any(score <= 0 for score in etf_momentum_score[:4])
str_VAA_agg = ""
etf_VAA_agg = ""

if has_negative_values:
  str_VAA_agg = "## 공격형 자산 중 0 이하 발생! 모멘텀 스코어 가장 높은 안전 자산 매수 필요\n"
  etf_VAA_agg = "## 추천 안전 자산: " + VAA_agg[etf_momentum_score.index(max(etf_momentum_score[4:]))]
else:
  str_VAA_agg = "## 공격형 자산 모두 0 이상! 모멘텀 스코어 가장 높은 공격 자산 매수 필요\n"
  etf_VAA_agg = "## 추천 공격 자산: " + VAA_agg[etf_momentum_score.index(max(etf_momentum_score[:4]))]

display(Markdown(str_today[1:]))
display(Markdown(str_VAA_agg))
display(Markdown(etf_VAA_agg))

# csv에 자산 투자 조언 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:]}')
  f.write(f'{str_VAA_agg}')
  f.write(f'{etf_VAA_agg}')


## 2023-08-15 

## 공격형 자산 중 0 이하 발생! 모멘텀 스코어 가장 높은 안전 자산 매수 필요


## 추천 안전 자산: SHY